In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models_05-retrieval-model/nvidia_logo.png" style="width: 90px; float: right;">

# Building a Retrieval Model with Merlin Models

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container. 

In large scale recommender systems pipelines, the size of the item catalog (number of unique items) might be in the order of millions. At such scale, a typical setup is having two-stage pipeline, where a faster candidate retrieval model quickly extracts thousands of relevant items and a then a more powerful ranking model (i.e. with more features and more powerful architecture) ranks the top-k items that are going to be displayed to the user. For ML-based candidate retrieval model, as it needs to quickly score millions of items for a given user, a popular choices are models that can produce recommendation scores by just computing the dot product the user embeddings and item embeddings. Popular choices of such models are **Matrix Factorization**, which learns low-rank user and item embeddings, and the **Two-Tower architecture**, which is a neural network with two MLP towers where both user and item features are fed to generate user and item embeddings in the output.


### Dataset

In this notebook, we are building a Two-Tower model for Item Retrieval task using synthetic datasets that are mimicking the real [Ali-CCP: Alibaba Click and Conversion Prediction](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1) dataset.
### Learning objectives
- Preparing the data with NVTabular
- Training and evaluating Two-Tower model with Merlin Models
- Exporting the model for deployment
- Generating the top K recommendations from the trained model

### Importing Libraries

In [2]:
import os

import nvtabular as nvt
from nvtabular.ops import *
from merlin.models.utils.example_utils import workflow_fit_transform

from merlin.schema.tags import Tags

import merlin.models.tf as mm
from merlin.io.dataset import Dataset

import tensorflow as tf

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-22 21:56:31.343779: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-22 21:56:32.499520: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0
/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.

In [3]:
# disable INFO and DEBUG logging everywhere
import logging

logging.disable(logging.WARNING)

### Feature Engineering with NVTabular

Let's generate synthetic train and validation dataset objects.

In [4]:
from merlin.datasets.synthetic import generate_data

DATA_FOLDER = os.environ.get("DATA_FOLDER", "/workspace/data/")
NUM_ROWS = os.environ.get("NUM_ROWS", 1000000)
SYNTHETIC_DATA = eval(os.environ.get("SYNTHETIC_DATA", "True"))

if SYNTHETIC_DATA:
    train, valid = generate_data("aliccp-raw", int(NUM_ROWS), set_sizes=(0.7, 0.3))
else:
    train = nvt.Dataset(DATA_FOLDER + "/train/*.parquet")
    valid = nvt.Dataset(DATA_FOLDER + "/valid/*.parquet")

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [5]:
train = train.to_ddf().compute()
valid = valid.to_ddf().compute()

We keep only positive interactions where clicks==1 in the dataset.

In [6]:
train = train.loc[train['click']==1].reset_index(drop=True)
valid = valid.loc[valid['click']==1].reset_index(drop=True)

We can drop the target column since in this example we will only use positive interactions and then generate negative samples via negative sampling technique.

In [7]:
train = train.drop(['click', 'conversion'], axis=1)
valid = valid.drop(['click', 'conversion'], axis=1)

Create Dataset objects

In [8]:
train = Dataset(train)
valid = Dataset(valid)

Define output path for the processed parquet files

In [9]:
output_path = os.path.join(DATA_FOLDER, "processed")

In [10]:
category_temp_directory = os.path.join(DATA_FOLDER, "categories")
user_id = ["user_id"] >> Categorify(out_path=category_temp_directory) >> TagAsUserID()
item_id = ["item_id"] >> Categorify(out_path=category_temp_directory) >> TagAsItemID()

item_features = ["item_category", "item_shop", "item_brand"] >> Categorify(out_path=category_temp_directory) >> TagAsItemFeatures()

user_features = (
    [
        "user_shops",
        "user_profile",
        "user_group",
        "user_gender",
        "user_age",
        "user_consumption_2",
        "user_is_occupied",
        "user_geography",
        "user_intentions",
        "user_brands",
        "user_categories",
    ]
    >> Categorify(out_path=category_temp_directory)
    >> TagAsUserFeatures()
)

outputs = user_id + item_id + item_features + user_features

With `transform_aliccp` function, we can execute fit() and transform() on the raw dataset applying the operators defined in the NVTabular workflow pipeline above. The processed parquet files are saved to output_path.

In [11]:
from merlin.datasets.ecommerce import transform_aliccp

transform_aliccp((train, valid), output_path, nvt_workflow=outputs)

## Building a Two-Tower Model with Merlin Models

We will use Two-Tower Model for item retrieval task. Real-world large scale recommender systems have hundreds of millions of items (products) and users. Thus, these systems often composed of two stages: candidate generation (retrieval) and ranking (scoring the retrieved items). At candidate generation step, a subset of relevant items from large item corpus is retrieved. You can read more about two stage Recommender Systems here. In this example, we're going to focus on the retrieval stage.

A Two-Tower Model consists of item (candidate) and user (query) encoder towers. With two towers, the model can learn representations (embeddings) for queries and candidates separately. 

<img src="./images/TwoTower.png"  width="30%">

Image Adapted from: [Off-policy Learning in Two-stage Recommender Systems](https://dl.acm.org/doi/abs/10.1145/3366423.3380130)

We use the `schema` object to define our model.

In [12]:
train = Dataset(os.path.join(output_path, "train", "*.parquet"))
valid = Dataset(os.path.join(output_path, "valid", "*.parquet"))

Select features with user and item tags, and be sure to exclude target column.

In [13]:
schema = train.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])
train.schema = schema
valid.schema = schema

We can print out the feature column names.

In [14]:
schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name
0,user_id,"(Tags.CATEGORICAL, Tags.USER_ID, Tags.USER, Ta...",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.u...,661.0,61.0,0,660,user_id
1,item_id,"(Tags.CATEGORICAL, Tags.ITEM, Tags.ITEM_ID, Ta...",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.i...,657.0,61.0,0,656,item_id
2,item_category,"(Tags.CATEGORICAL, Tags.ITEM)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.i...,657.0,61.0,0,656,item_category
3,item_shop,"(Tags.CATEGORICAL, Tags.ITEM)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.i...,657.0,61.0,0,656,item_shop
4,item_brand,"(Tags.CATEGORICAL, Tags.ITEM)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.i...,657.0,61.0,0,656,item_brand
5,user_shops,"(Tags.CATEGORICAL, Tags.USER)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.u...,661.0,61.0,0,660,user_shops
6,user_profile,"(Tags.CATEGORICAL, Tags.USER)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.u...,46.0,16.0,0,45,user_profile
7,user_group,"(Tags.CATEGORICAL, Tags.USER)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.u...,9.0,16.0,0,8,user_group
8,user_gender,"(Tags.CATEGORICAL, Tags.USER)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.u...,3.0,16.0,0,2,user_gender
9,user_age,"(Tags.CATEGORICAL, Tags.USER)",int64,False,False,None,0.0,0.0,0.0,/workspace/data/categories/categories/unique.u...,6.0,16.0,0,5,user_age


We expect the label names to be empty.

In [15]:
label_names = schema.select_by_tag(Tags.TARGET).column_names
label_names

[]

### Negative sampling
Many datasets for recommender systems contain implicit feedback with logs of user interactions like clicks, add-to-cart, purchases, music listening events, rather than explicit ratings that reflects user preferences over items. To be able to learn from implicit feedback, we use the general (and naive) assumption that the interacted items are more relevant for the user than the non-interacted ones.
In Merlin Models we provide some scalable negative sampling algorithms for the Item Retrieval Task. In particular, we use in this example the in-batch sampling algorithm which uses the items interacted by other users as negatives within the same mini-batch.

### Building the Model

Now, let's build our Two-Tower model. In a nutshell, we aggregate all user features to feed in user tower and feed the item features to the item tower. Then we compute the positive score by multiplying the user embedding with the item embedding and sample negative items (read more about negative sampling [here](https://openreview.net/pdf?id=824xC-SgWgU) and [here](https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7)), whose item embeddings are also multiplied by the user embedding. Then we apply the loss function on top of the positive and negative scores.

In [16]:
# create user schema using USER tag
user_schema = schema.select_by_tag(Tags.USER)
# create user (query) tower input block
user_inputs = mm.InputBlockV2(user_schema)
# create user (query) encoder block
query = mm.Encoder(user_inputs, mm.MLPBlock([128, 64]))

# create item schema using ITEM tag
item_schema = schema.select_by_tag(Tags.ITEM)
# create item (candidate) tower input block
item_inputs = mm.InputBlockV2(item_schema)
# create item (candidate) encoder block
candidate = mm.Encoder(item_inputs, mm.MLPBlock([128, 64]))


Build the model class.

In [17]:
model = mm.TwoTowerModelV2(query, candidate)

Note that in the `TwoTowerModelV2` function we did not set `negative_samplers` arg, that means it is set to None. In that case, Two-tower model is trained with contrastive learning and `in-batch` negative sampling strategy.

**Metrics:**

The following information retrieval metrics are used to compute the Top-10 accuracy of recommendation lists containing all items:

- **Normalized Discounted Cumulative Gain (NDCG@10)**: NDCG accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-k items.

- **Recall@10**: Also known as HitRate@n when there is only one relevant item in the recommendation list. Recall just verifies whether the relevant item is among the top-n items.

We need to initialize the dataloaders.

In [18]:
model.compile(optimizer="adam", run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10)])
model.fit(train, validation_data=valid, batch_size=4096, epochs=2)

Epoch 1/2
86/86 [==============================] - 12s 49ms/step - loss: 8.2953 - recall_at_10: 0.0274 - ndcg_at_10: 0.0258 - regularization_loss: 0.0000e+00 - loss_batch: 8.2732 - val_loss: 8.2914 - val_recall_at_10: 0.0311 - val_ndcg_at_10: 0.0311 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 7.8196
Epoch 2/2
86/86 [==============================] - 3s 35ms/step - loss: 8.2949 - recall_at_10: 0.0324 - ndcg_at_10: 0.0324 - regularization_loss: 0.0000e+00 - loss_batch: 8.2728 - val_loss: 8.2918 - val_recall_at_10: 0.0252 - val_ndcg_at_10: 0.0247 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 7.8190


## Exporting Retrieval Models

So far we have trained and evaluated our Retrieval model. Now, the next step is to deploy our model and generate top-K recommendations given a user (query). We can efficiently serve our model by indexing the trained item embeddings into an **Approximate Nearest Neighbors (ANN)** engine. Basically, for a given user query vector, that is generated passing the user features into user tower of retrieval model, we do an ANN search query to find the ids of nearby item vectors, and at serve time, we score user embeddings over all indexed top-K item embeddings within the ANN engine.

In doing so, we need to export
 
- user (query) tower
- item and user features
- item embeddings

#### Save and Load User (query) tower

We are able to save the user tower model as a TF model to disk. The user tower model is needed to generate a user embedding vector when a user feature vector <i>x</i> is fed into that model.

In [19]:
query_tower = model.query_encoder
query_tower.save(os.path.join(output_path, 'query_model_merlin'))

query_tower_loaded = tf.keras.models.load_model(os.path.join(output_path, 'query_model_merlin'))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(


#### Extract and save User features

With `unique_rows_by_features` utility function we can easily extract both unique user and item features tables as cuDF dataframes. Note that for user features table, we use `USER` and `USER_ID` tags.

In [20]:
from merlin.models.utils.dataset import unique_rows_by_features

user_features = (
    unique_rows_by_features(train, Tags.USER, Tags.USER_ID).compute().reset_index(drop=True)
)

In [21]:
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories
0,1,1,1,1,1,1,1,1,1,1,1,1
1,2,2,1,1,1,1,1,1,1,2,2,2
2,3,3,1,1,1,1,1,1,1,3,3,3
3,4,4,1,1,1,1,1,1,1,4,4,4
4,5,5,1,1,1,1,1,1,1,5,5,5


In [22]:
user_features.shape

(660, 12)

In [23]:
# save to disk
user_features.to_parquet(os.path.join(DATA_FOLDER, "user_features.parquet"))

#### Geneate Query embeddings for entire user catalog

In [24]:
queries = model.query_embeddings(Dataset(user_features, schema=schema), batch_size=1024, index=Tags.USER_ID)
query_embs_df = queries.compute(scheduler="synchronous").reset_index()

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(


In [27]:
query_embs_df.head()

,user_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,1,0.000000,0.008196,0.137290,0.0,0.092979,0.074278,0.005642,0.072807,0.000000,...,0.0,0.059925,0.0,0.153969,0.000000,0.000000,0.0,0.153310,0.066789,0.006385
1,2,0.179585,0.017029,0.000000,0.0,0.068811,0.131977,0.013031,0.072382,0.000579,...,0.0,0.108048,0.0,0.165289,0.000000,0.000000,0.0,0.032833,0.132182,0.101511
2,3,0.000000,0.030797,0.193472,0.0,0.089610,0.119547,0.039946,0.132592,0.000000,...,0.0,0.000000,0.0,0.119517,0.020564,0.022465,0.0,0.000000,0.082197,0.048806
3,4,0.070694,0.000000,0.213158,0.0,0.120921,0.075420,0.064782,0.035988,0.000000,...,0.0,0.111238,0.0,0.037444,0.000000,0.000000,0.0,0.068493,0.015514,0.121884
4,5,0.045353,0.023883,0.200193,0.0,0.145634,0.155438,0.000000,0.048233,0.000000,...,0.0,0.017794,0.0,0.189992,0.000000,0.004357,0.0,0.150954,0.055630,0.033601


#### Extract and save Item features

In [28]:
item_features = (
    unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID).compute().reset_index(drop=True)
)

In [29]:
item_features.head()

,item_id,item_category,item_shop,item_brand
0,1,1,1,1
1,2,2,2,2
2,3,3,3,3
3,4,4,4,4
4,5,5,5,5


In [30]:
# save to disk
item_features.to_parquet(os.path.join(DATA_FOLDER, "item_features.parquet"))

#### Extract and save Item embeddings

In [32]:
item_embs = model.candidate_embeddings(Dataset(item_features, schema=schema), batch_size=1024, index=Tags.ITEM_ID)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [34]:
item_embs_df = item_embs.compute(scheduler="synchronous")

In [35]:
item_embs_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
item_id,,,,,,,,,,,,,,,,,,,,,
1,0.063365,0.024812,0.210963,0.059610,0.00000,0.000000,0.040406,0.060498,0.041382,0.023024,...,0.000000,0.000000,0.000000,0.120593,0.0,0.000000,0.026491,0.079933,0.000000,0.056472
2,0.052140,0.044581,0.092967,0.060396,0.00000,0.000000,0.000168,0.024269,0.067360,0.115878,...,0.000000,0.036687,0.000000,0.046636,0.0,0.022346,0.000000,0.035753,0.109223,0.097380
3,0.160856,0.019256,0.160834,0.121810,0.00000,0.000000,0.044383,0.014302,0.199200,0.049542,...,0.000000,0.097169,0.066355,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.042171
4,0.066548,0.039639,0.121373,0.052577,0.00000,0.009728,0.071900,0.040178,0.041055,0.104913,...,0.035017,0.070771,0.022091,0.051197,0.0,0.000000,0.000000,0.097419,0.019362,0.067408
5,0.052963,0.040653,0.009650,0.053568,0.00000,0.000000,0.031105,0.000000,0.000000,0.113457,...,0.000000,0.045452,0.000000,0.094761,0.0,0.000000,0.000000,0.221804,0.000000,0.036065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,0.062518,0.034162,0.065428,0.047812,0.00000,0.000413,0.045096,0.017163,0.027124,0.079644,...,0.000000,0.016099,0.000000,0.084044,0.0,0.000000,0.006223,0.062150,0.069739,0.029321
653,0.032423,0.057289,0.099077,0.062227,0.00000,0.004452,0.041264,0.058788,0.066255,0.023188,...,0.000000,0.007465,0.035758,0.085552,0.0,0.005834,0.002472,0.060991,0.021136,0.022467
654,0.018789,0.024882,0.123488,0.027976,0.00000,0.000000,0.009125,0.036854,0.046543,0.049662,...,0.000000,0.000000,0.001378,0.115050,0.0,0.000000,0.000000,0.021774,0.009610,0.023257


In [37]:
# save to disk
item_embs_df.to_parquet(os.path.join(DATA_FOLDER, "item_embeddings.parquet"))

That's it. You have learned how to train and evaluate your Two-Tower retrieval model, and then how to export the required components to be able to deploy this model to generate recommendations. In order to learn more on serving a model to [Triton Inference Server](https://github.com/triton-inference-server/server), please explore the examples in the [Merlin](https://github.com/NVIDIA-Merlin/Merlin) and [Merlin Systems](https://github.com/NVIDIA-Merlin/systems) repos.

### Generate top-K recommendations

In [38]:
# Top-K evaluation
candidate_features = unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID)
topk_model = model.to_top_k_encoder(candidate_features, k=20, batch_size=128)

# we can set `metrics` param in the `compile(), if we want
topk_model.compile(run_eagerly=False)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


This is doing dot product for entire catalog. By default it is brute force.

In [40]:
loader = mm.Loader(train, batch_size=1024, transform=mm.ToTarget(schema, "item_id"))

metrics = topk_model.evaluate(loader, return_dict=True)

342/342 [==============================] - 6s 17ms/step - loss: 0.6536 - recall_at_10: 0.1439 - mrr_at_10: 0.0567 - ndcg_at_10: 0.0771 - map_at_10: 0.0567 - precision_at_10: 0.0144 - regularization_loss: 0.0000e+00 - loss_batch: 0.6531


In [41]:
metrics

{'loss': 0.6535949110984802,
 'recall_at_10': 0.14391019940376282,
 'mrr_at_10': 0.056715019047260284,
 'ndcg_at_10': 0.07706044614315033,
 'map_at_10': 0.056715019047260284,
 'precision_at_10': 0.014391022734344006,
 'regularization_loss': 0.0,
 'loss_batch': 0.5503028035163879}

Let's generate top-K (k=20 in our exp) recommendations for a given batch of 8 samples. The to_top_k_encoder() method uses the index provided in the item (candidate) features dataset as the identifiers. The recommended top 20 item ids are returned below. Note that these are encoded item ids not the original ids.

In [49]:
loader = mm.Loader(train, batch_size=8, shuffle=False)
batch =next(iter(loader))

In [50]:
topk_model(batch[0])

TopKPrediction(scores=<tf.Tensor: shape=(8, 20), dtype=float32, numpy=
array([[0.17605971, 0.17392597, 0.16495053, 0.16066968, 0.14865187,
        0.14713697, 0.14579962, 0.1416155 , 0.13890888, 0.13834275,
        0.13757811, 0.13751204, 0.13697278, 0.13640182, 0.13630816,
        0.13614711, 0.13577895, 0.13574389, 0.13496579, 0.13378012],
       [0.22762711, 0.2199027 , 0.20631751, 0.20208453, 0.20002982,
        0.19945292, 0.19743744, 0.19651861, 0.19314197, 0.19239512,
        0.19178613, 0.19140297, 0.19060233, 0.18965288, 0.18962705,
        0.18597771, 0.18574229, 0.1853667 , 0.18432552, 0.18418384],
       [0.2112518 , 0.206018  , 0.19263393, 0.18393432, 0.17986958,
        0.17832413, 0.17248616, 0.17239921, 0.17128868, 0.17109446,
        0.16921255, 0.1683428 , 0.16595232, 0.16594367, 0.16573189,
        0.16436909, 0.16300151, 0.16252977, 0.16134106, 0.16098835],
       [0.18074025, 0.15562603, 0.15481664, 0.1547099 , 0.15262872,
        0.15194924, 0.15189493, 0.14974773